In [136]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [137]:
df = pd.read_csv("DFF_result.csv")

# Clean data.

# Pulse width values for Pass cases & Current percentage values for Fail cases. 
df.loc[df['fail'] == 'pass', 'Junction Values'] = df.loc[df['fail'] == 'pass', 'Pulse width info']
df.loc[df['fail'] == 'fail', 'Junction Values'] = df.loc[df['fail'] == 'fail', 'Current percentage']


# One-hot encode pass/fail.
drop_col = ['cus', 'real_flux', 'Faulty Junction', 'Pulse magnitude info', 'Phase state info', 'Pulse width info', 'Current percentage' ]
df = df.drop(drop_col, axis=1)
# df = pd.get_dummies(df, columns=["fail"], prefix=["Outcome"])

df.head()

,config,pattern,MC,fail,real_output,gold_output,Junction Values
0,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.0,0.04,-0.04",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6..."
1,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,-0.04",fail,"{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7..."
2,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,0.01",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.79, 'B01RX2': 0.69, 'B01TX1': 0.6..."
3,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.06,0.05",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6..."
4,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.03",pass,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.77, 'B01RX2': 0.67, 'B01TX1': 0.6..."


In [147]:
# Rule 1: Initial Bias is between 60%-80%. 
# Rule 2: Junction 4 bias <55%. 


# Create rule 1, rule 2, pass/fail truth table.
for val in df['Junction Values']:
    val = eval(str(val))
    if 0.6 <= (val['B01RX1']) <= 0.8:
        df['Rule 1'] = 1
    else:
        df['Rule 1'] = 0
    if val['B4'] <0.55:
        df['Rule 2'] = 1
    else:
        df['Rule 2'] = 0

columns = df.columns.tolist()
columns.append(columns.pop(3))
df["fail"] = df["fail"].replace({"pass": 1, "fail": 0})
df = df[columns]

df.head(20)

# condition6 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 0])
# print(df[condition6])

,config,pattern,MC,Rule 1,Rule 2,fail,real_output,gold_output,Junction Values
0,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.0,0.04,-0.04",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6..."
1,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,-0.04",1,1,0,"{'n300': '0000000000000000', 'n200': '00100000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.85, 'B01RX2': 0.73, 'B01TX1': 0.7..."
2,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.01,0.01",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.79, 'B01RX2': 0.69, 'B01TX1': 0.6..."
3,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.06,0.05",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6..."
4,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.03",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.77, 'B01RX2': 0.67, 'B01TX1': 0.6..."
5,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,-0.07,0.05",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.75, 'B01RX2': 0.66, 'B01TX1': 0.6..."
6,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.01,-0.01",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.81, 'B01RX2': 0.7, 'B01TX1': 0.68..."
7,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.01,0.01",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.78, 'B01RX2': 0.69, 'B01TX1': 0.6..."
8,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.02,-0.03",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.83, 'B01RX2': 0.72, 'B01TX1': 0.6..."
9,and_spb_0_dff_spa_0_and,"0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1","-0.01,0.03,-0.02",1,1,1,"{'n300': '0000000000000000', 'n200': '00010000...","{'n300': '0000000000000000', 'n200': '00010000...","{'B01RX1': 0.82, 'B01RX2': 0.71, 'B01TX1': 0.6..."


In [160]:
# Create buckets.
condition0 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 0]).all(axis=1)
condition0_df = df[condition0]
print(len(condition0_df))

condition1 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 1]).all(axis=1)
condition1_df = df[condition1]
print(len(condition1_df))

condition2 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 0]).all(axis=1)
condition2_df = df[condition2]
print(len(condition2_df))

condition3 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 1]).all(axis=1)
condition3_df = df[condition3]
print(len(condition3_df))

condition4 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 0]).all(axis=1)
condition4_df = df[condition4]
print(len(condition4_df))

condition5 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 1]).all(axis=1)
condition5_df = df[condition5]
print(len(condition5_df))

condition6 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 0]).all(axis=1)
condition6_df = df[condition6]
print(len(condition6_df))

condition7 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 1]).all(axis=1)
condition7_df = df[condition7]
print(len(condition7_df))

0
0
0
0
0
0
16713
23288


In [161]:
import Quine_McCluskey_Functions as qm